In [4]:
import os 
from tqdm import tqdm 
from copy import deepcopy
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from optuna import Trial
from optuna.samplers import TPESampler
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import joblib
import pickle as pkl
from copy import deepcopy

In [7]:
train_fold = pd.read_csv('data/train_fold.csv', dtype={'text_id' : 'object'})
dict_ = {}
for i in range(len(train_fold)):
    dict_[str(train_fold.loc[i, 'text_id'])] = i

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn import svm
from catboost import CatBoostRegressor

eval_list = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

coefs = []

df_sum = pd.DataFrame([])
for k in eval_list:
    data = pd.DataFrame([])
    for idx, i in enumerate(os.listdir('predict')):   # 각 모델
        try:
            df = pd.read_csv(os.path.join('predict', i, 'oof_data.csv'), index_col=0)
            data['{}'.format(i)] = df[k]
        except:
            pass
        
#     for i in range(1, 13):
#         with open("predict/oof_df.pkl ({})/oof_df.pkl".format(i), "rb") as f:
#             df = pkl.load(f)
#         dict__ = []
#         for j in range(len(df)):
#             try:
#                 l = dict_[df.loc[j, 'text_id']]
#                 dict__.append(l)
#             except:
#                 ValueError()
#         df['idx'] = dict__
#         df.sort_values(by='idx', inplace=True)
#         df.reset_index(drop=True, inplace=True)
#         data[i] = df['pred_'+k]
        
    train = data.values
    ans = pd.read_csv('data/train_fold.csv')[k].values
    linear = LinearRegression()
    linear.fit(train, ans)
    prediction = linear.predict(train)

    print('{} : {}'.format(k, mean_squared_error(prediction, ans, squared=False)))
    coefs.append(linear.coef_)

cohesion : 0.47327529420322045
syntax : 0.4370771137258687
vocabulary : 0.4058784493645005
phraseology : 0.44559806140272423
grammar : 0.4635767862220229
conventions : 0.43840399747582937


In [12]:
coefs

[array([-0.02468827,  0.01437969, -0.16997332, -0.06378136,  0.09925593,
        -0.14434502,  0.25566551,  0.04314837, -0.00287905,  0.00157988,
         0.21722764,  0.17301293,  0.01158325,  0.37524524,  0.2222383 ,
         0.02657244, -0.17524487,  0.12026351]),
 array([-0.00554599, -0.09864255, -0.03283034,  0.01100232,  0.08877053,
        -0.44237157,  0.42066285,  0.14919479, -0.05189426, -0.02122462,
         0.31019634,  0.13965195,  0.08109633,  0.22936185,  0.20586317,
        -0.03187159, -0.0620047 ,  0.09124441]),
 array([ 0.01133957,  0.00102318,  0.2249395 ,  0.10911379,  0.11591557,
        -0.12086344,  0.14706209, -0.07191999, -0.02380568, -0.13426628,
         0.1688741 ,  0.02553323,  0.11637734,  0.07900849,  0.2870512 ,
        -0.0325324 , -0.05061934,  0.13387843]),
 array([-0.01358336,  0.14455209, -0.00932055,  0.06291627,  0.11673717,
        -0.21839524,  0.21565667, -0.04944162,  0.0283303 ,  0.11075765,
         0.29320124, -0.27259406,  0.04855632,  0.

In [ ]:
length = int(len(df_sum) * .8)
train1 = df_sum[:length][['cohesion', 'cohesion_mean', 'cohesion_median']].values
ans1 = train_fold['cohesion'][:length].values
train2 = df_sum[length:][['cohesion', 'cohesion_mean', 'cohesion_median']].values
ans2 = train_fold['cohesion'][length:].values
local_score = 0.4713640057952513

In [ ]:
train = df_sum[['syntax', 'syntax_mean', 'syntax_median']].values
ans = train_fold['syntax'].values

linear = LinearRegression()
linear.fit(train, ans)
prediction = linear.predict(train)
prediction

In [ ]:
print(mean_squared_error(prediction, ans, squared=False))
linear.coef_

In [ ]:


#     coefs.append(linear.coef_)
save = []
save2 = []
def objective(trial: Trial) -> float:

#     params_lgb = {
#         "random_state": 42,
#         "verbosity": 1,
#         "learning_rate": trial.suggest_float('lr', .05, .1),
#         "n_estimators": 1000,
#         "objective": "regression",
#         "metric": "rmse",
#         "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
#         "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
#         "max_depth": trial.suggest_int("max_depth", 2, 20),
# #             "num_leaves": trial.suggest_int("num_leaves", 2, 256),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
#         "subsample": trial.suggest_float("subsample", 0.3, 1.0),
#         "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#         "max_bin": trial.suggest_int("max_bin", 200, 500),
#         'num_boost_round' : trial.suggest_int("max_bin", 100, 1000)
#     }

    model = XGBRegressor(**params)

    model.fit(train1, ans1, eval_set=[(train2, ans2)], verbose=0)
    pred1 = model.predict(train1)
    pred2 = model.predict(train2)
    loss1 = mean_squared_error(pred1, ans1) ** .5        # train
    loss2 = mean_squared_error(pred2, ans2) ** .5        # valid

    if (local_score > loss2): 
        save.append(loss1)
        save2.append(loss2)

    return loss2

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=100)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

print('save : ', save)
print('save2 : ', save2)
print('local_score', local_score)
break

In [ ]:
data_save = deepcopy(data)
ans = pd.read_csv('data/train_fold.csv')[k].values
train_data = data[:3500]
valid_data = data[3500:]
train_ans = ans[:3500]
valid_ans = ans[3500:]
train_ans = np.expand_dims(train_ans, axis=1)
valid_ans = np.expand_dims(valid_ans, axis=1)

In [ ]:
import os 
from tqdm import tqdm 
from copy import deepcopy
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from optuna import Trial
from optuna.samplers import TPESampler
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import joblib

In [ ]:
answer = pd.read_csv('data/train_fold.csv')
index = answer['fold']
list_ = ['sentences_count',
'spaces_count',
'chars_excl_spaces_count',
'punctuations_count',
'count_words',
'duplicates_count',
'stop_words_count',
'noun_phrase_count',]

eval_list = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

cnt = 0
for k in eval_list:
    for j in range(4):
        
        train_data = pd.DataFrame([])
        valid_data = pd.DataFrame([])

        train_label = answer[k][index != j]
        valid_label = answer[k][index == j]

        for idx, i in enumerate(os.listdir('predict')):
            path = os.path.join('predict', i, 'oof_data.csv')
            local_data = pd.read_csv(path, index_col=0)
            train_feature = local_data[index != j][k]
            valid_feature = local_data[index == j][k]
            train_data['{}'.format(idx)] = train_feature
            valid_data['{}'.format(idx)] = valid_feature

        for i in list_:
            appending = answer[i]
            train_appending = appending[index != j]
            valid_appending = appending[index == j]
            train_data[i] = train_appending
            valid_data[i] = valid_appending
            
        for i in list_:
            if i == 'chars_excl_spaces_count':
                continue
            else:
                train_data[i] = train_data[i] / train_data['chars_excl_spaces_count']

        numeric_features = []                      ########################################## HAVE TO BE CHANGE
        numeric_transformer = StandardScaler()
        categorical_features = ['0', '1', '2', '3', '4', '5', '6']
        # categorical_transformer = OneHotEncoder(categories='auto') # categories='auto' : just for ignoring warning messages
        preprocessor = ColumnTransformer(
            transformers = [ ('num', numeric_transformer, numeric_features),
                ('passthrough', 'passthrough', categorical_features)])
        preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])
        
        
        train_data = preprocessor_pipe.fit_transform(train_data)
        valid_data = preprocessor_pipe.fit_transform(valid_data)
        
        def objective(trial: Trial) -> float:

            params_lgb = {
                "random_state": 42,
                "verbosity": -1,
                "learning_rate": trial.suggest_float('lr', .05, .1),
                "n_estimators": 1000,
                "objective": "multiclass",
                "metric": "rmse",
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
                "max_depth": trial.suggest_int("max_depth", 2, 20),
#                 "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
                "subsample": trial.suggest_float("subsample", 0.3, 1.0),
                "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
                "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                "max_bin": trial.suggest_int("max_bin", 200, 500),
                'num_boost_round' : trial.suggest_int("max_bin", 100, 1000)
            }
            model = LGBMRegressor(**params_lgb)

            model.fit(train_data, train_label, eval_set=[(train_data, train_label), (valid_data, valid_label)], early_stopping_rounds=100, verbose=False)
            pred = model.predict(valid_data)
            loss = mean_squared_error(pred, valid_label) ** .5

            return loss
        sampler = TPESampler(seed=42)
        study = optuna.create_study(
            study_name="lgbm_parameter_opt",
            direction="minimize",
            sampler=sampler,
        )
        study.optimize(objective, n_trials=100)
        print("Best Score:", study.best_value)
        print("Best trial:", study.best_trial.params)
        
        print(study.best_params)
        
        model = LGBMRegressor(**study.best_params)
        model.fit(train_data, train_label)
        joblib.dump(model, 'lgb/lgb_{}.pkl'.format(cnt))
        cnt += 1

In [ ]:
import torch
torch.save({'model' : final}, 'final.pt')

In [ ]:
list___ = []
for i in range(0, 24, 4):
    list_ = final_[i:i+4]
    k = np.array(list_).sum() / 4
    list___.append(k)

In [ ]:
np.array(list___).sum() / 6

In [ ]:

pred = model.predict(valid_feature)
ans = valid_label


In [ ]:
score = mean_squared_error(train_data[:, 0], train_label, squared=False)
score